# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract folder

In [25]:
img_size=128
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size),
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


interactive(children=(Select(description='x', options=('scratch', 'proj'), value='scratch'), Output()), _dom_c…

In [26]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20200911_083711_bsize64_spec_test_128_nospec', '2020…

In [28]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20210202_061803_bsize256_scale0.1_deterministic_off


In [29]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

In [30]:
# main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/scaling_runs/20201031_205304_bsize512_scaling_2_8'

## Extract metrics info from log file

In [31]:
def f_extract_info(fname):
    '''
    Module to extract information from out.log files of Lbann training
    Reads in file name
    '''
    strg_lst=['objective','d_real','d_fake','gen','spec_loss','run time','mini-batch']
    keys=['training_'+strg for strg in strg_lst]
    dict1={}
    for category in ['training','validation']:
        for strg in strg_lst:
            try: 
                key=category+'_'+strg
                cmd='grep "{0}" {1} | grep "{2}"'.format(category,fname,strg)
        #         print(cmd)
                op1=sp.check_output(cmd,shell=True).decode('utf-8').split('\n')
                obj=np.array([strg.split(':')[-1] for strg in op1 if strg])
                dict1[key]=obj
            except Exception as e:
                print(e)
                dict1[key]=np.nan
    
    df=pd.DataFrame([])
    key_lst=['training_objective', 'training_d_real', 'training_d_fake', 'training_gen', 'training_spec_loss','validation_objective', 'validation_d_real', 'validation_d_fake', 'validation_gen','validation_spec_loss']
    col_list=['train_obj','train_dreal','train_dfake','train_gen','train_spec','val_obj','val_dreal','val_dfake','val_gen','val_spec']
    for col,key in zip(col_list,key_lst):
        try: 
            df[col]=dict1[key].astype(np.float)
        except: pass

    ### Need to remove the trailing 's' in the timings
    for col,key in zip(['train_time','val_time'],['training_run time','validation_run time']):
        df[col]=np.array([i[:-1] for i in dict1[key]]).astype(np.float)

    for col,key in zip(['train_batch_stats','val_batch_stats'],['training_mini-batch','validation_mini-batch']):
        df[col]=dict1[key]
        
    return df

def f_plot_metrics(df,col_list=['train_obj']):
    '''
    Plot multiple columns of the dataframe
    '''
    plt.figure()
    
    marker_lst=('o','*','H','D','.','x')
    marker=itertools.cycle(marker_lst)
    for col in col_list: plt.plot(df[col],linestyle='',marker=next(marker),label=col)
    plt.legend()
    plt.xlabel('Epoch')
    

In [32]:
strg=main_dir+'/out.log'
df_metrics=f_extract_info(glob.glob(strg)[0])

In [33]:
df_metrics.head()

,train_obj,train_dreal,train_dfake,train_gen,train_spec,val_obj,val_dreal,val_dfake,val_gen,val_spec,train_time,val_time,train_batch_stats,val_batch_stats
0,15.70320,1.219360,1.275580,11.89480,13.1346,9.72420,2.616520,0.124483,5.805400,11.7781,419.047,5.82597,"0.476522s mean, 9.02995s max, 0.0266426s min,...","0.0594051s mean, 0.500174s max, 0.0212752s mi..."
1,6.16578,0.495612,0.483422,4.01696,11.6979,30.56000,11.834300,0.180882,17.390200,11.5463,399.467,7.62367,"0.454238s mean, 6.55073s max, 0.0262884s min,...","0.0777488s mean, 1.6343s max, 0.0164776s min,..."
2,5.98767,0.352452,0.325682,4.14563,11.6391,4.48296,0.069835,2.882260,0.372353,11.5852,392.013,5.70072,"0.445768s mean, 6.92787s max, 0.0262816s min,...","0.0581269s mean, 0.429303s max, 0.0163983s mi..."
3,5.45568,0.371758,0.362550,3.56512,11.5625,3.23870,0.083976,1.568680,0.428946,11.5710,413.820,5.27831,"0.470575s mean, 7.37449s max, 0.0286093s min,...","0.0538159s mean, 0.531878s max, 0.0161677s mi..."
4,4.75274,0.422011,0.418764,2.75837,11.5359,4.33674,0.069739,2.945630,0.174320,11.4705,432.904,5.38392,"0.492289s mean, 7.94243s max, 0.034194s min, ...","0.054895s mean, 0.421384s max, 0.015976s min,..."


In [34]:
col_list=df_metrics.columns[~df_metrics.columns.str.endswith('stats')]
interact_manual(f_plot_metrics,col_list=SelectMultiple(options=col_list),df=fixed(df_metrics))


interactive(children=(SelectMultiple(description='col_list', options=('train_obj', 'train_dreal', 'train_dfake…

<function __main__.f_plot_metrics(df, col_list=['train_obj'])>

## Extract data from stored dataframe

In [35]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [36]:
print(df.shape)
df.tail()


(1758, 22)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
1753,19,17530,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.012314,0.003623,0.064946,0.080883,0.009492,7.234464,...,27.924074,256,6,0,"[1.0324120250151863, 0.4833778653314031, 0.047...","[0.0061994722428055015, 0.002886563777617123, ...","[-1.031746031584782, -0.3099415213241682, 0.30...","[86054.84338229336, 58904.55309350601, 40782.5...","[51522.429441339336, 26735.41809674116, 14631....",19-17530
1754,19,17540,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.015703,0.019646,0.018856,0.054204,0.002606,5.655101,...,28.438981,256,4,0,"[1.078009645441199, 0.46300683653468133, 0.040...","[0.005345835552838916, 0.0024995417832572634, ...","[-1.031746031584782, -0.3099415213241682, 0.30...","[68567.71611077792, 49736.280925710125, 34590....","[42832.1456446733, 23094.90884673777, 13859.98...",19-17540
1755,19,17550,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.098205,0.034606,0.001630,0.134441,0.066222,4.942041,...,26.836938,256,2,0,"[0.8939091884305197, 0.5619064409914444, 0.044...","[0.0050044978948510874, 0.00229512806880783, 0...","[-1.031746031584782, -0.3099415213241682, 0.30...","[75560.26050882507, 53452.91650881411, 39056.3...","[43394.08534986266, 23170.520471767315, 15346....",19-17550
1756,19,17560,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.042530,0.030196,0.020480,0.093207,0.021573,5.434245,...,31.128085,256,3,0,"[0.9920302282529874, 0.5097364161387988, 0.041...","[0.005039394229785307, 0.002341409166684236, 0...","[-1.031746031584782, -0.3099415213241682, 0.30...","[63710.87812915363, 46878.58477273731, 34701.4...","[34604.56291157187, 20053.32546088902, 13774.6...",19-17560
1757,19,17570,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.042433,0.042363,0.001921,0.086718,0.011488,4.413333,...,32.528637,256,0,0,"[1.029681603286208, 0.4913095865480288, 0.0380...","[0.004374303473791922, 0.0020661710993399823, ...","[-1.031746031584782, -0.3099415213241682, 0.30...","[59203.955795026384, 44448.797060642035, 32287...","[27651.378560401874, 17199.38868901689, 11489....",19-17570


In [37]:
df.columns

Index(['epoch', 'step', 'img_type', 'fname', 'chi_1a', 'chi_1b', 'chi_1c',
       'chi_1', 'chi_2', 'chi_imgvar', 'chi_spec1', 'chi_spec2', 'chi_spec3',
       'num_imgs', 'num_large', 'num_vlarge', 'hist_val', 'hist_err',
       'hist_bin_centers', 'spec_val', 'spec_sdev', 'label'],
      dtype='object')

### View best epochs

#### Locations with best chi_sqr

In [38]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]


,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
1414,16,14140,0.000267,0.002252,0.069976,0.072495,0.000030,6.270127,0.842327,6.543771e+01
957,10,9570,0.001671,0.000481,0.042050,0.044202,0.000103,6.278294,0.348262,3.151294e+01
1548,17,15480,0.015591,0.020328,0.000484,0.036403,0.007812,5.272540,1.212303,8.300044e+01
1224,13,12240,0.001359,0.008560,0.000710,0.010629,0.000116,5.931152,0.574117,5.160315e+01
1414,16,14140,0.000267,0.002252,0.069976,0.072495,0.000030,6.270127,0.842327,6.543771e+01
3,0,30,1.138960,0.108666,0.003864,1.251490,0.794701,0.072941,39318.138066,8.939555e+07
1332,15,13320,0.009182,0.009819,0.044881,0.063882,0.004912,5.616035,0.264044,2.362047e+01
1475,16,14750,0.001227,0.006010,0.064428,0.071665,0.000102,6.712430,0.385929,1.758401e+01


In [39]:
### Plot chi-sqr values
# _=df_sliced.plot(x="step", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

In [40]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

# df[df.num_vlarge>10].tail()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

### View best steps

In [41]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [42]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [43]:
df_sliced=w.result
# df_sliced

### Pick best steps

In [44]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1b',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_spec1',head=4,display_flag=False).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.04732605457061089, 'chi_spec1': 0.8274391860788071}
[0, 19]
{'chi_1': 0.1795470502820763, 'chi_spec1': 4.3121110527938376}
[0, 19]
{'chi_1': 0.04732605457061089, 'chi_spec1': 0.8274391860788071}
[ 7200  7730  9400  9570 10950 11040 11810 12240 13390 13540]


In [45]:
# best_step=[3430,  6460,  7215,  9610, 11725, 14635, 9475]
# best_step=np.arange(40130,40135).astype(int)

In [46]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(10, 22)
[(8, 7200), (8, 7730), (10, 9400), (10, 9570), (12, 10950), (12, 11040), (13, 11810), (13, 12240), (15, 13390), (15, 13540)]


In [48]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
720,8-7200,0.076392,0.543510,29.615042,0.002419,0.000562,0.073411,0.000615,7.276323,58.662930,8,7200
773,8-7730,0.028670,0.287699,28.817490,0.005814,0.018835,0.004021,0.001581,5.668081,22.756355,8,7730
940,10-9400,0.012939,0.680204,26.299114,0.002214,0.002509,0.008216,0.000190,5.795412,64.586702,10,9400
957,10-9570,0.044202,0.348262,28.516661,0.001671,0.000481,0.042050,0.000103,6.278294,31.512938,10,9570
1095,12-10950,0.011643,0.489623,26.840275,0.002727,0.006557,0.002358,0.000723,5.430217,41.068094,12,10950
1104,12-11040,0.034542,0.303276,24.486601,0.013615,0.009950,0.010976,0.008824,5.536650,31.562543,12,11040
1181,13-11810,0.014349,0.692907,24.005744,0.003461,0.007678,0.003210,0.001418,5.445718,69.578829,13,11810
1224,13-12240,0.010629,0.574117,25.791356,0.001359,0.008560,0.000710,0.000116,5.931152,51.603151,13,12240
1339,15-13390,0.044513,0.280854,26.753791,0.019133,0.002511,0.022869,0.013029,6.482032,29.028585,15,13390
1354,15-13540,0.024597,0.306160,29.364236,0.008355,0.006398,0.009844,0.005247,5.876878,26.482876,15,13540


## Plot pixel intensity and spectrum

In [49]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [50]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))

    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)
            
        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)
            plt.legend()
            
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=100)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')

    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
    
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [51]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('8-7200', '8-7730', '10-9400', '10-95…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [52]:
df[(df.step>8000)& (df.step<10000)].plot(kind='line',x='step',y=['chi_1','chi_spec1'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='step'>

## View image block

In [ ]:
def f_get_img(df,step):
    df_temp=df[(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,8020)
# f_plot_grid(img_arr[:18],cols=6,fig_size=(10,5))

In [ ]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,5))

### Plotting spread of spectrum of input data

In [ ]:

# plt.figure()

# idxs=np.random.randint(0,s_val.shape[0],size=50)
# arr=s_val[idxs]
# Pk = f_batch_spectrum(arr)
# k=np.arange(Pk.shape[1])
# for count,i in enumerate(Pk):
#     plt.plot(i)
    
# Pk = f_batch_spectrum(img_arr)
# mean,std = np.mean(Pk, axis=0),np.std(Pk, axis=0)/np.sqrt(Pk.shape[0])
# plt.plot(k, mean, 'k:',label='generated image')


# plt.legend()

# plt.ylabel(r'$P(k)$')
# plt.xlabel(r'$k$')
# plt.title('Power Spectrum: 50 input images')
# plt.yscale('log')



In [ ]:
main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/'
f1=main_dir+'20201208_165658_bsize256_scale0.1_deterministic_on/dump_outs/trainer0/model0/sgd.training.epoch.1.step.1000_gen_img_instance1_activation_output0.npy'
a1=np.load(f1)[:,0,:,:]
f2=main_dir+'20201209_055955_bsize256_scale0.1_deterministic_on/dump_outs/trainer0/model0/sgd.training.epoch.1.step.1000_gen_img_instance1_activation_output0.npy'
a2=np.load(f2)[:,0,:,:]

print(a1.shape)
print(a2.shape)


In [ ]:
# f_pixel_intensity(a1)
# f_pixel_intensity(a2)

f_compare_pixel_intensity([a1,a2],label_lst=['img1','img2'],bkgnd_arr=[],log_scale=True, normalize=True, mode='avg',bins=25, hist_range=None)

In [ ]:
df_best.columns

In [ ]:
f_invtransform(df_best.iloc[0].hist_bin_centers)


In [ ]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use



In [ ]:
bins